In [1]:
class Sentiment:
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"

In [2]:
class Review:
    def __init__(self,text,score):
        self.text=text
        self.score=score
        self.sentiment=self.getSentiment()
    def getSentiment(self):
        if self.score<=2:
            return Sentiment.NEGATIVE
        elif self.score>2 and self.score<4:
            return Sentiment.NEUTRAL
        else:
            return Sentiment.POSITIVE

In [3]:
import random
class ReviewContainer:
    def __init__(self,reviews):
        self.reviews=reviews ########## "reviews" collection of "Review" objects
    def get_text(self):
        return [x.text for x in self.reviews]
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
    def evenly_distribute(self):
        negative=list(filter(lambda x:x.sentiment==Sentiment.NEGATIVE,self.reviews))
        positive=list(filter(lambda x:x.sentiment==Sentiment.POSITIVE,self.reviews))
        nutral=list(filter(lambda x:x.sentiment==Sentiment.NEUTRAL,self.reviews))
        positive_chunk=positive[:len(negative)+1]
        self.reviews=negative+positive_chunk+nutral
        random.shuffle(self.reviews)

In [4]:
import json
file_name = './data/sentiment/books_small_10000.json'
reviews=[]

with open(file_name) as f:
    for line in f: ###### go line by line
        review=json.loads(line)
        reviews.append(Review(review['reviewText'],review['overall']))
        


In [5]:
from sklearn.model_selection import train_test_split
training, test = train_test_split(reviews, test_size=.2, random_state=42)
train_container = ReviewContainer(training)
test_container = ReviewContainer(test)

In [8]:
train_container.evenly_distribute()
test_container.evenly_distribute()

train_x=train_container.get_text()
train_y=train_container.get_sentiment()

test_x=test_container.get_text()
test_y=test_container.get_sentiment()

print(train_y.count(Sentiment.POSITIVE))
print(train_y.count(Sentiment.NEGATIVE))
print(train_y.count(Sentiment.NEUTRAL))

print(test_y.count(Sentiment.POSITIVE))
print(test_y.count(Sentiment.NEGATIVE))
print(test_y.count(Sentiment.NEUTRAL))

514
513
783
132
131
195


In [13]:
from sklearn.feature_extraction.text import CountVectorizer ### unweighted not so much better
from sklearn.feature_extraction.text import TfidfVectorizer ### weighted and better than CountVectorizer

vectorizer=TfidfVectorizer()

train_x_vectors=vectorizer.fit_transform(train_x)
test_x_vectors=vectorizer.transform(test_x)

print(train_x[0])
print(train_x_vectors[0].toarray())

print(test_x[0])
print(test_x_vectors[0].toarray())


OMG Melissa has done it again!  What a way to end this beautiful Love series!!  Although I was really nervous going into the reading because it was my first M/M book...I was beautifully surprised!  This book is powerfully written!  It completely captures you and doesn't let go until the very end!  Not all stories have beautiful lessons woven throughout it's pages, but this one does....love is unconditional and available to all!!  Warning however, LOTS of Kleenex is needed! I love Melissa Collins and her stories!  Please read her books...I promise you will not be disappointed!!
[[0. 0. 0. ... 0. 0. 0.]]
Story was ok. Love scenes were prolonged. I got stuck in a couple of chapters for lack of interest. Overall, good book.
[[0. 0. 0. ... 0. 0. 0.]]


In [14]:
from sklearn import svm

clf_svm = svm.SVC(kernel='linear')

clf_svm.fit(train_x_vectors, train_y)

clf_svm.predict(test_x_vectors[0])

array(['NEUTRAL'], dtype='<U8')